In [28]:
from astroquery.mast import Observations

In [ ]:
obsTable = Observations.query_criteria(project='HST',filters='F160W',instrument_name='WFC3/IR')

In [30]:
products = Observations.get_product_list(obsTable[:100])

In [31]:
filtered_products = Observations.filter_products(products,mrp_only=False,productSubGroupDescription='DRZ')

In [33]:
Observations.enable_s3_hst_dataset()

INFO: Using the S3 HST public dataset [astroquery.mast.core]
INFO: See Request Pricing in https://aws.amazon.com/s3/pricing/ for details [astroquery.mast.core]
INFO: If you have not configured boto3, follow the instructions here: https://boto3.readthedocs.io/en/latest/guide/configuration.html [astroquery.mast.core]


In [34]:
s3_urls = Observations.get_hst_s3_uris(filtered_products)

In [35]:
s3_urls[:10]

['s3://stpubdata/hst/public/icde/icde43l0q/icde43l0q_drz.fits',
 's3://stpubdata/hst/public/icde/icde43l1q/icde43l1q_drz.fits',
 's3://stpubdata/hst/public/icde/icde43l2q/icde43l2q_drz.fits',
 's3://stpubdata/hst/public/icde/icde43l3q/icde43l3q_drz.fits',
 's3://stpubdata/hst/public/icde/icde43l4q/icde43l4q_drz.fits',
 's3://stpubdata/hst/public/icdu/icdu01npq/icdu01npq_drz.fits',
 's3://stpubdata/hst/public/icdu/icdu36ecq/icdu36ecq_drz.fits',
 's3://stpubdata/hst/public/icdx/icdx1ygxq/icdx1ygxq_drz.fits',
 's3://stpubdata/hst/public/icdx/icdx1zxhq/icdx1zxhq_drz.fits',
 's3://stpubdata/hst/public/icdx/icdx2axuq/icdx2axuq_drz.fits']

In [36]:
import boto3
import json

In [37]:
# Auth to create a Lambda function (credentials are picked up from .boto file in home directory)
session = boto3.Session(profile_name='lambda-invoker')
client = session.client('lambda', region_name='us-east-1')

In [25]:
# Create a Lambda function.
response = client.create_function(
    FunctionName='Astrometry9d66002',
    Runtime='python2.7',
    Role='arn:aws:iam::793754315137:role/AstropyLambdaRole',
    Handler='process.handler',
    Code={
        'S3Bucket': 'dsmo-lambda-functions',
        'S3Key': '9d66002.zip'
    },
    Description='Testing Lambda astrometry from JuypyterLab!',
    Timeout=300,
    MemorySize=1024,
    Publish=True
)

In [38]:
# Loop through the URLs for the data on S3
for url in s3_urls:
    fits_s3_key = url.replace("s3://stpubdata/", "")
    event = {
          'fits_s3_key': fits_s3_key,
          'fits_s3_bucket': 'stpubdata',
          's3_output_bucket': 'dsmo-lambda-test-outputs'
        }
    
    # Invoke Lambda function
    response = client.invoke(
        FunctionName='Astrometry9d66002',
        InvocationType='Event',
        LogType='Tail',
        Payload=json.dumps(event)
)